# Test Unitarios

In [5]:
# TESTS UNITARIOS - EXTRACCIÓN

from typing import Dict, List
from pyspark.sql.functions import col

print("="*70)
print("TESTS UNITARIOS")
print("="*70)

tests_passed = 0
tests_failed = 0

def test(condition, name):
    global tests_passed, tests_failed
    if condition:
        tests_passed += 1
        print(f"✅ {name}")
    else:
        tests_failed += 1
        print(f"❌ {name}")

# COPIAR FUNCIONES A TESTEAR

def normalize_record(record: Dict):
    """Ajusta el registro al esquema definido"""
    return {
        "id": record.get("id"),
        "title": record.get("title"),
        "url": record.get("url"),
        "image_url": record.get("image_url"),
        "news_site": record.get("news_site"),
        "summary": record.get("summary"),
        "published_at": record.get("published_at"),
        "updated_at": record.get("updated_at"),
        "featured": record.get("featured"),
        "launches": [str(x) for x in record.get("launches") or []],
        "events": [str(x) for x in record.get("events") or []]
    }

def deduplicate(records):
    """Elimina duplicados usando ID como clave primaria"""
    unique = {}
    for r in records:
        if r.get("id") not in unique:
            unique[r["id"]] = r
    return list(unique.values())

class SpaceflightAPI:
    """Cliente Spaceflight News API"""
    def __init__(self, base_url: str):
        self.base_url = base_url
        import requests
        self.session = requests.Session()

BASE_URL = "https://api.spaceflightnewsapi.net/v4"

# TESTS DE FUNCIONES

print("\nFunciones:")

# Test normalize_record
input_rec = {"id": 1, "title": "Test", "launches": [1, 2], "events": None}
result = normalize_record(input_rec)
test(result["id"] == 1, "normalize_record - id")
test(result["launches"] == ["1", "2"], "normalize_record - launches")
test(result["events"] == [], "normalize_record - events None")

# Test deduplicate
records = [{"id": 1}, {"id": 2}, {"id": 1}]
dedup = deduplicate(records)
test(len(dedup) == 2, "deduplicate - elimina duplicados")

# Test API init
api = SpaceflightAPI(BASE_URL)
test(api.base_url == BASE_URL, "SpaceflightAPI - inicialización")

# TESTS DE DATOS BRONZE

print("\nDatos Bronze:")

# Existencia de tablas
for table in ['bronze_articles', 'bronze_blogs', 'bronze_reports']:
    try:
        spark.table(table)
        test(True, f"{table} existe")
    except:
        test(False, f"{table} existe")

# Schema correcto
df = spark.table('bronze_articles')
test('id' in df.columns, "bronze_articles - tiene columna id")
test('title' in df.columns, "bronze_articles - tiene columna title")
test('summary' in df.columns, "bronze_articles - tiene columna summary")
test('published_at' in df.columns, "bronze_articles - tiene columna published_at")

# Sin IDs nulos
null_count = df.filter(col("id").isNull()).count()
test(null_count == 0, "bronze_articles - sin IDs nulos")

# Sin duplicados
total = df.count()
distinct = df.select("id").distinct().count()
test(total == distinct, "bronze_articles - sin duplicados")

# Tiene datos
test(total > 0, "bronze_articles - contiene datos")

# ============================================
# RESUMEN
# ============================================

print("\n" + "="*70)
total = tests_passed + tests_failed
print(f"Total: {total} | ✅ {tests_passed} | ❌ {tests_failed}")
print(f"Tasa éxito: {tests_passed/total*100:.0f}%")
print("="*70)

if tests_failed > 0:
    print(f"{tests_failed} test(s) fallaron")
else:
    print("Todos los tests pasaron")

StatementMeta(, be7062c6-4c95-4efd-a830-ef276403441b, 7, Finished, Available, Finished, False)

TESTS UNITARIOS

Funciones:
✅ normalize_record - id
✅ normalize_record - launches
✅ normalize_record - events None
✅ deduplicate - elimina duplicados
✅ SpaceflightAPI - inicialización

Datos Bronze:
✅ bronze_articles existe
✅ bronze_blogs existe
✅ bronze_reports existe
✅ bronze_articles - tiene columna id
✅ bronze_articles - tiene columna title
✅ bronze_articles - tiene columna summary
✅ bronze_articles - tiene columna published_at
✅ bronze_articles - sin IDs nulos
✅ bronze_articles - sin duplicados
✅ bronze_articles - contiene datos

Total: 15 | ✅ 15 | ❌ 0
Tasa éxito: 100%
Todos los tests pasaron


In [ ]:
class SpaceflightAPI:
    """
    Cliente Spaceflight News API
    - Paginación con campo 'next'
    - Retry con backoff exponencial
    - Rate limit
    """

    def __init__(self, base_url: str):
        self.base_url = base_url
        self.session = requests.Session()

    @retry(stop=stop_after_attempt(3), wait=wait_exponential(min=2, max=10))
    def _request(self, url: str) -> Dict:
        response = self.session.get(url, timeout=30)
        response.raise_for_status()
        return response.json()

    def fetch_paginated(self, endpoint: str):
        """
        Extrae todos los registros de un endpoint paginado.
        """
        records = []
        url = f"{self.base_url}/{endpoint}"

        logging.info(f"Iniciando extracción: {endpoint}")

        while url:
            data = self._request(url)

            page = data.get("results", [])
            records.extend(page)

            logging.info(
                f"{endpoint}: {len(page)} registros | Total: {len(records)}"
            )

            url = data.get("next")
            time.sleep(RATE_LIMIT_SECONDS)

        logging.info(f"Extracción completa {endpoint}: {len(records)} registros")
        return records

    def fetch_info(self) -> Dict:
        logging.info("Extrayendo metadata API")
        return self._request(f"{self.base_url}/info")

In [1]:
# ============================================
# TESTS UNITARIOS - EXTRACCIÓN
# ============================================

print("="*70)
print("TESTS UNITARIOS")
print("="*70)

tests_passed = 0
tests_failed = 0

def test(condition, name):
    global tests_passed, tests_failed
    if condition:
        tests_passed += 1
        print(f"✅ {name}")
    else:
        tests_failed += 1
        print(f"❌ {name}")

# ============================================
# TESTS DE FUNCIONES
# ============================================

print("\n📋 Funciones:")

# Test normalize_record
input_rec = {"id": 1, "title": "Test", "launches": [1, 2], "events": None}
result = normalize_record(input_rec)
test(result["id"] == 1, "normalize_record - id")
test(result["launches"] == ["1", "2"], "normalize_record - launches")
test(result["events"] == [], "normalize_record - events None")

# Test deduplicate
records = [{"id": 1}, {"id": 2}, {"id": 1}]
dedup = deduplicate(records)
test(len(dedup) == 2, "deduplicate - elimina duplicados")

# Test API init
api = SpaceflightAPI(BASE_URL)
test(api.base_url == BASE_URL, "SpaceflightAPI - inicialización")

# ============================================
# TESTS DE DATOS BRONZE
# ============================================

print("\n📋 Datos Bronze:")

# Existencia de tablas
for table in ['bronze_articles', 'bronze_blogs', 'bronze_reports']:
    try:
        spark.table(table)
        test(True, f"{table} existe")
    except:
        test(False, f"{table} existe")

# Schema correcto
df = spark.table('bronze_articles')
test('id' in df.columns, "bronze_articles - tiene columna id")
test('title' in df.columns, "bronze_articles - tiene columna title")
test('ingestion_date' in df.columns, "bronze_articles - tiene ingestion_date")

# Sin IDs nulos
null_count = df.filter(col("id").isNull()).count()
test(null_count == 0, "bronze_articles - sin IDs nulos")

# Sin duplicados
total = df.count()
distinct = df.select("id").distinct().count()
test(total == distinct, "bronze_articles - sin duplicados")

# Tiene datos
test(total > 0, "bronze_articles - contiene datos")

# ============================================
# RESUMEN
# ============================================

print("\n" + "="*70)
total = tests_passed + tests_failed
print(f"Total: {total} | ✅ {tests_passed} | ❌ {tests_failed}")
print(f"Tasa éxito: {tests_passed/total*100:.0f}%")
print("="*70)

StatementMeta(, be7062c6-4c95-4efd-a830-ef276403441b, 3, Finished, Available, Finished, False)

TESTS UNITARIOS

📋 Funciones:


NameError: name 'normalize_record' is not defined